In [43]:
import subprocess
import altair as alt
import pandas as pd

In [44]:
# In order to run clingo command in the terminal and capture the output
def run_clingo(puzzle):
    result = subprocess.run(['clingo', "instances/"+puzzle, 'programs/programNonOpt.lp', '--stats'], 
                            capture_output=True, text=True)
    output = result.stdout
    return output

In [45]:
def analyze_output(output):
    lines = output.split('\n')
    stats = {}
    for line in lines:
        if "CPU Time" in line:
            stats["time"] = float((line.split(":")[1].strip()).split("s")[0])
        elif "Rules" in line:
            stats["rules"] = int((line.split(":")[1].strip()).split(" ")[0])
        elif "Atoms" in line:
            stats["atoms"] = int((line.split(":")[1].strip()).split(" ")[0])

    return stats

In [46]:
def classify_puzzle(stats):
    if stats["time"] < 0.2:
        return "Very easy"
    elif stats["time"] < 0.5:
        return "Easy"
    elif stats["time"] < 1:
        return "Medium"
    elif stats["time"] < 2:
        return "Hard"
    else:
        return "Harder"

In [47]:
def analyze_puzzles(puzzles):
    results = []
    i = 1
    for puzzle in puzzles:
        output = run_clingo(puzzle)
        stats = analyze_output(output)
        difficulty = classify_puzzle(stats)
        stats["difficulty"] = difficulty
        stats["puzzle"] = puzzle
        stats["size"] = i
        i += 1
        results.append(stats)

    return results

In [48]:
puzzles = ["param_gen1.db", "param_gen2.db","param_gen3.db", "param_gen4.db", "param_gen5.db",
            "param_gen6.db", "param_gen7.db", "param_gen8.db",
              "param_gen9.db"]
puzzles_results = analyze_puzzles(puzzles)
df = pd.DataFrame(puzzles_results)
print(df)

    time    rules  atoms difficulty         puzzle  size
0  0.003      729    532  Very easy  param_gen1.db     1
1  0.010     8292   1115  Very easy  param_gen2.db     2
2  0.034    32950   2235  Very easy  param_gen3.db     3
3  0.112    83118   3811  Very easy  param_gen4.db     4
4  0.221   174716   5321       Easy  param_gen5.db     5
5  0.332   310101   7491       Easy  param_gen6.db     6
6  0.560   500880  10039     Medium  param_gen7.db     7
7  0.931   750441  13235     Medium  param_gen8.db     8
8  1.355  1087946  16311       Hard  param_gen9.db     9


In [49]:
chart = alt.Chart(df).mark_line().encode(
    alt.X('size', type='ordinal', title='Puzzle size'),
    alt.Y('time', type='quantitative', title='Time taken (s)'),
).properties(title='Time taken to solve puzzles').interactive()
chart.show()

alt.Chart(...)